In [19]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import lightgbm as lgb
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [18]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [3]:
df_train=pd.read_csv('train.csv')

df_test=pd.read_csv('test.csv')

In [4]:
df_train.isnull().mean()

Year_Factor                  0.000000
State_Factor                 0.000000
building_class               0.000000
facility_type                0.000000
floor_area                   0.000000
                               ...   
direction_peak_wind_speed    0.551909
max_wind_speed               0.542287
days_with_fog                0.604512
site_eui                     0.000000
id                           0.000000
Length: 64, dtype: float64

In [5]:
df_test.isnull().mean()

Year_Factor                  0.000000
State_Factor                 0.000000
building_class               0.000000
facility_type                0.000000
floor_area                   0.000000
                               ...   
direction_max_wind_speed     0.883565
direction_peak_wind_speed    0.883565
max_wind_speed               0.883565
days_with_fog                0.939413
id                           0.000000
Length: 63, dtype: float64

In [14]:
def entrena_lgb(data,test,features,categorical,target):

    kfold=GroupKFold(n_splits=6)


    i=1

    r=[]
    
    pred_test=np.zeros(len(test))

    importancias=pd.DataFrame()

    importancias['variable']=features
    
    
    cat_ind=[features.index(x) for x in categorical if x in features]
    
    dict_cat={}
    
    categorical_numerical = data[categorical].dropna().select_dtypes(include=np.number).columns.tolist()
    
    categorical_transform=[x for x in categorical if x not in categorical_numerical]
    
    for l in categorical_transform:
        le = preprocessing.LabelEncoder()
        le.fit(list(data[l].dropna())+list(test[l].dropna()))

        dict_cat[l]=le

        data.loc[~data[l].isnull(),l]=le.transform(data.loc[~data[l].isnull(),l])
        test.loc[~test[l].isnull(),l]=le.transform(test.loc[~test[l].isnull(),l])
        
        

    for train_index,test_index in kfold.split(data,data[target],data['Year_Factor']):

        lgb_data_train = lgb.Dataset(data.loc[train_index,features].values,data.loc[train_index,target].values)
        lgb_data_eval = lgb.Dataset(data.loc[test_index,features].values,data.loc[test_index,target].values, reference=lgb_data_train)

        params = {
            'task': 'train',
            #'boosting_type': 'gbdt',
            #'objective': 'regression',
            #'metric': { 'rmse'},
            #'num_iterations':5000,
            #'max_bin':5395,
            "max_depth":12,
            "num_leaves":52,
            'learning_rate': 0.873791759,
        #"min_child_samples": 100,
            #'feature_fraction': 0.9,
         #"bagging_freq":1,
            #'bagging_fraction': 0.9,
            #"lambda_l1":10,
            #"lambda_l2":10,
           # "scale_pos_weight":30,
            #'min_data_per_group':500,

            'verbose': 1    
        }




        modelo = lgb.train(params,
                           lgb_data_train,
                           num_boost_round=95100,
                           valid_sets=lgb_data_eval,
                           early_stopping_rounds=100,
                           verbose_eval=25,
                           categorical_feature=cat_ind
                          )

        importancias['gain_'+str(i)]=modelo.feature_importance(importance_type="gain")


        data.loc[test_index,'estimator']=modelo.predict(data.loc[test_index,features].values, num_iteration=modelo.best_iteration)
        
        pred_test=pred_test+modelo.predict(test[features].values, num_iteration=modelo.best_iteration)

        print ("Fold_"+str(i))
        a= (mean_squared_error(data.loc[test_index,target],data.loc[test_index,'estimator']))**0.5
        r.append(a)
        print (a)
        print ("")

        i=i+1
        
    for l in categorical_transform:

            data.loc[~data[l].isnull(),l]=dict_cat[l].inverse_transform(data.loc[~data[l].isnull(),l].astype(int))
            
            test.loc[~test[l].isnull(),l]=dict_cat[l].inverse_transform(test.loc[~test[l].isnull(),l].astype(int))
            
    importancias["gain_avg"]=importancias[["gain_1","gain_2","gain_3","gain_4","gain_5"]].mean(axis=1)
    importancias=importancias.sort_values("gain_avg",ascending=False).reset_index(drop=True)
    
    pred_test=(pred_test/6)
    
    
    oof=(mean_squared_error(data[target],data['estimator']))**0.5
    
    print (oof)
    print ("mean: "+str(np.mean(np.array(r))))
    print ("std: "+str(np.std(np.array(r))))
    
    dict_resultados={}
    
    dict_resultados['importancias']=importancias
    
    dict_resultados['predicciones']=pred_test
    
    
    
    return dict_resultados

In [15]:
no_usar=['site_eui','id']

target='site_eui'

categorical=['Year_Factor','State_Factor','building_class','facility_type']

features=[x for x in df_train.columns if x not in no_usar]

In [20]:
dict_resultados=entrena_lgb(data=df_train,test=df_test,features=features,categorical=categorical,target=target)

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3760
[LightGBM] [Info] Number of data points in the train set: 53308, number of used features: 62
[LightGBM] [Info] Start training from score 83.961575
Training until validation scores don't improve for 100 rounds
[25]	valid_0's l2: 2637.54

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')



[50]	valid_0's l2: 2740.66
[75]	valid_0's l2: 2804.81
[100]	valid_0's l2: 2956.99
Early stopping, best iteration is:
[2]	valid_0's l2: 1976.18
Fold_1
44.454289737098094



C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

[50]	valid_0's l2: 2447.36
[75]	valid_0's l2: 2454.64
[100]	valid_0's l2: 2462.83
Early stopping, best iteration is:
[1]	valid_0's l2: 2291.88
Fold_5
47.873604881402024



C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007331 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3973
[LightGBM] [Info] Number of data points in the train set: 73640, number of used features: 62
[LightGBM] [Info] Start training from score 82.261660
Training until validation scores don't improve for 100 rounds
[25]	valid_0's l2: 4259.21

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')



[50]	valid_0's l2: 4237.74
[75]	valid_0's l2: 4248.98
[100]	valid_0's l2: 4287.26
Early stopping, best iteration is:
[8]	valid_0's l2: 3520.09
Fold_6
59.33030562241879

45.079124511799584
mean: 47.52079775406876
std: 6.085053238924374


In [21]:
dict_resultados['importancias']

,variable,gain_1,gain_2,gain_3,gain_4,gain_5,gain_6,gain_avg
0,facility_type,3.693611e+07,3.942270e+07,5.324571e+07,5.622216e+07,49811362.0,5.765547e+07,4.712761e+07
1,energy_star_rating,2.913359e+07,3.155403e+07,3.886339e+07,4.651688e+07,39242554.0,4.336900e+07,3.706209e+07
2,year_built,3.787851e+06,3.644358e+06,1.526926e+07,1.847699e+07,2065724.0,1.009398e+07,8.648836e+06
3,floor_area,3.428341e+06,4.996962e+06,1.495246e+07,1.771676e+07,497694.0,1.278669e+07,8.318442e+06
4,State_Factor,1.817088e+06,1.501267e+06,1.447687e+06,2.164111e+06,1681423.0,2.253189e+06,1.722315e+06
...,...,...,...,...,...,...,...,...
57,Year_Factor,0.000000e+00,0.000000e+00,4.915270e+04,4.568641e+04,0.0,3.607430e+04,1.896782e+04
58,days_below_20F,0.000000e+00,0.000000e+00,0.000000e+00,4.903723e+04,0.0,0.000000e+00,9.807446e+03
59,days_above_100F,0.000000e+00,0.000000e+00,1.555090e+04,6.962300e+03,0.0,0.000000e+00,4.502640e+03
60,days_below_0F,0.000000e+00,0.000000e+00,0.000000e+00,1.598450e+04,0.0,2.907470e+04,3.196900e+03


In [22]:
dict_resultados['importancias']['variable'].tolist()

['facility_type',
 'energy_star_rating',
 'year_built',
 'floor_area',
 'State_Factor',
 'ELEVATION',
 'january_min_temp',
 'days_with_fog',
 'building_class',
 'june_min_temp',
 'precipitation_inches',
 'july_avg_temp',
 'snowdepth_inches',
 'june_avg_temp',
 'october_avg_temp',
 'november_max_temp',
 'november_avg_temp',
 'july_min_temp',
 'march_avg_temp',
 'april_avg_temp',
 'may_avg_temp',
 'august_min_temp',
 'september_avg_temp',
 'january_avg_temp',
 'september_max_temp',
 'october_min_temp',
 'snowfall_inches',
 'march_min_temp',
 'february_avg_temp',
 'max_wind_speed',
 'direction_max_wind_speed',
 'avg_temp',
 'september_min_temp',
 'october_max_temp',
 'november_min_temp',
 'april_min_temp',
 'january_max_temp',
 'february_min_temp',
 'may_min_temp',
 'december_max_temp',
 'days_below_30F',
 'june_max_temp',
 'march_max_temp',
 'december_avg_temp',
 'february_max_temp',
 'april_max_temp',
 'direction_peak_wind_speed',
 'cooling_degree_days',
 'december_min_temp',
 'days_abo

In [23]:
temp=dict_resultados['importancias']

features_selected=temp['variable'].tolist()[0:4]

In [24]:
dict_resultados_2=entrena_lgb(data=df_train,test=df_test,features=features_selected,categorical=categorical,target=target)

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1780:

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001127 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 580
[LightGBM] [Info] Number of data points in the train set: 53308, number of used features: 4
[LightGBM] [Info] Start training from score 83.961575
Training until validation scores don't improve for 100 rounds
[25]	valid_0's l2: 1692.51
[50]	valid_0's l2: 1660.73
[75]	valid_0's l2: 1617.89
[100]	valid_0's l2: 1596.2
[125]	valid_0's l2: 1605.53
[150]	valid_0's l2: 1596.21
[175]	valid_0's l2: 1586.88
[200]	valid_0's l2: 1600.02
[225]	valid_0's l2: 1600.46
[250]	valid_0's l2: 1608.24
[275]	valid_0's l2: 1600.77
Early stopping, best iteration is:
[177]	valid_0's l2: 1582.07
Fold_1
39.77524483156612

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001297 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 582
[Li

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1780:

[50]	valid_0's l2: 1101.78
[75]	valid_0's l2: 1025.87
[100]	valid_0's l2: 976.845
[125]	valid_0's l2: 935.548
[150]	valid_0's l2: 903.244
[175]	valid_0's l2: 880.323
[200]	valid_0's l2: 872.55
[225]	valid_0's l2: 853.486
[250]	valid_0's l2: 837.495
[275]	valid_0's l2: 821.364
[300]	valid_0's l2: 815.05
[325]	valid_0's l2: 804.459
[350]	valid_0's l2: 794.978
[375]	valid_0's l2: 785.643
[400]	valid_0's l2: 783.515
[425]	valid_0's l2: 777.981
[450]	valid_0's l2: 774.72
[475]	valid_0's l2: 773.924
[500]	valid_0's l2: 769.34
[525]	valid_0's l2: 766.392
[550]	valid_0's l2: 764.951
[575]	valid_0's l2: 759.444
[600]	valid_0's l2: 760.067
[625]	valid_0's l2: 757.628
[650]	valid_0's l2: 756.666
[675]	valid_0's l2: 755.622
[700]	valid_0's l2: 753.127
[725]	valid_0's l2: 751.49
[750]	valid_0's l2: 751.709
[775]	valid_0's l2: 751.029
[800]	valid_0's l2: 751.156
[825]	valid_0's l2: 751.369
[850]	valid_0's l2: 751.144
[875]	valid_0's l2: 751.114
[900]	valid_0's l2: 749.142
[925]	valid_0's l2: 747.284

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1780:

...

[50]	valid_0's l2: 2143.51
[75]	valid_0's l2: 2077.24
[100]	valid_0's l2: 2031.72
[125]	valid_0's l2: 1994.6
[150]	valid_0's l2: 1971.85
[175]	valid_0's l2: 1967.84
[200]	valid_0's l2: 1952.5
[225]	valid_0's l2: 1927.72
[250]	valid_0's l2: 1916.62
[275]	valid_0's l2: 1906.81
[300]	valid_0's l2: 1895.99
[325]	valid_0's l2: 1886.28
[350]	valid_0's l2: 1871.86
[375]	valid_0's l2: 1861.14
[400]	valid_0's l2: 1853.47
[425]	valid_0's l2: 1846.15
[450]	valid_0's l2: 1839.93
[475]	valid_0's l2: 1836.51
[500]	valid_0's l2: 1828.96
[525]	valid_0's l2: 1825.24
[550]	valid_0's l2: 1822.66
[575]	valid_0's l2: 1820.71
[600]	valid_0's l2: 1816.26
[625]	valid_0's l2: 1813.47
[650]	valid_0's l2: 1810.7
[675]	valid_0's l2: 1798.99
[700]	valid_0's l2: 1786.27
[725]	valid_0's l2: 1783.19
[750]	valid_0's l2: 1781.65
[775]	valid_0's l2: 1780.37
[800]	valid_0's l2: 1777.77
[825]	valid_0's l2: 1777.15
[850]	valid_0's l2: 1774.47
[875]	valid_0's l2: 1771.93
[900]	valid_0's l2: 1771.61
[925]	valid_0's l2: 1765.

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1780:

[50]	valid_0's l2: 1804.79
[75]	valid_0's l2: 1751.22
[100]	valid_0's l2: 1707.91
[125]	valid_0's l2: 1687.73
[150]	valid_0's l2: 1673.64
[175]	valid_0's l2: 1648.6
[200]	valid_0's l2: 1635.14
[225]	valid_0's l2: 1624.55
[250]	valid_0's l2: 1610.66
[275]	valid_0's l2: 1600.2
[300]	valid_0's l2: 1586.52
[325]	valid_0's l2: 1577.38
[350]	valid_0's l2: 1563.86
[375]	valid_0's l2: 1551.52
[400]	valid_0's l2: 1552.46
[425]	valid_0's l2: 1549.15
[450]	valid_0's l2: 1538.91
[475]	valid_0's l2: 1535.54
[500]	valid_0's l2: 1539.05
[525]	valid_0's l2: 1538.53
[550]	valid_0's l2: 1532.92
[575]	valid_0's l2: 1528.49
[600]	valid_0's l2: 1514.2
[625]	valid_0's l2: 1516.9
[650]	valid_0's l2: 1511.75
[675]	valid_0's l2: 1510.81
[700]	valid_0's l2: 1503.77
[725]	valid_0's l2: 1503.31
[750]	valid_0's l2: 1499.42
[775]	valid_0's l2: 1497.23
[800]	valid_0's l2: 1493.41
[825]	valid_0's l2: 1490.3
[850]	valid_0's l2: 1493.59
[875]	valid_0's l2: 1489.11
[900]	valid_0's l2: 1485.04
[925]	valid_0's l2: 1482.96

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:1780:


[50]	valid_0's l2: 3158.69
[75]	valid_0's l2: 3086.23
[100]	valid_0's l2: 2969.58
[125]	valid_0's l2: 2910.64
[150]	valid_0's l2: 2834.91
[175]	valid_0's l2: 2805.21
[200]	valid_0's l2: 2765.51
[225]	valid_0's l2: 2757.61
[250]	valid_0's l2: 2771.72
[275]	valid_0's l2: 2758.47
[300]	valid_0's l2: 2740.39
[325]	valid_0's l2: 2729.35
[350]	valid_0's l2: 2734.94
[375]	valid_0's l2: 2722.52
[400]	valid_0's l2: 2739.48
[425]	valid_0's l2: 2727.57
[450]	valid_0's l2: 2707.89
[475]	valid_0's l2: 2705.65
[500]	valid_0's l2: 2683.78
[525]	valid_0's l2: 2674.42
[550]	valid_0's l2: 2674.89
[575]	valid_0's l2: 2672.54
[600]	valid_0's l2: 2665.76
[625]	valid_0's l2: 2654.69
[650]	valid_0's l2: 2638.58
[675]	valid_0's l2: 2630.31
[700]	valid_0's l2: 2636.99
[725]	valid_0's l2: 2637.18
[750]	valid_0's l2: 2633.18
[775]	valid_0's l2: 2625.38
[800]	valid_0's l2: 2617.55
[825]	valid_0's l2: 2620.13
[850]	valid_0's l2: 2616.7
[875]	valid_0's l2: 2615.46
[900]	valid_0's l2: 2606.83
[925]	valid_0's l2: 26

In [25]:
dict_resultados_2['importancias']

,variable,gain_1,gain_2,gain_3,gain_4,gain_5,gain_6,gain_avg
0,facility_type,4.363831e+07,4.879127e+07,5.654507e+07,5.721875e+07,6.291575e+07,6.313175e+07,5.382183e+07
1,energy_star_rating,4.247192e+07,4.983772e+07,5.136190e+07,5.904194e+07,6.202707e+07,6.102984e+07,5.294811e+07
2,floor_area,3.035035e+07,4.390589e+07,4.786867e+07,4.607578e+07,4.949362e+07,5.231202e+07,4.353886e+07
3,year_built,3.107508e+07,3.891868e+07,4.330132e+07,4.526724e+07,4.666862e+07,4.865817e+07,4.104619e+07


In [26]:
df_train.groupby([ 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION']).size()

State_Factor  building_class  facility_type  floor_area  year_built  energy_star_rating  ELEVATION
0             0               2              13491.0     1967.0      24.0                9.1          1
                                             21600.0     1924.0      95.0                45.7         1
                                                                     97.0                45.7         1
                                             25500.0     1925.0      0.0                 2.4          1
                                             37000.0     1949.0      93.0                2.4          1
                                                                                                     ..
6             1               27             794056.0    1962.0      88.0                18.3         1
                                             821141.0    1963.0      84.0                21.3         1
                                             878176.0    2005.0      

In [27]:
df_train.groupby([ 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION']).size().value_counts()

1    46287
2      941
3       68
4       14
5        2
6        1
dtype: int64

In [28]:
df_train.groupby([ 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built',  'ELEVATION']).size().value_counts()

1     42239
2      7928
3      3386
4       802
5       407
6        36
7         9
8         6
12        2
10        2
9         2
18        1
17        1
dtype: int64

In [29]:
features_selected_2=list(features_selected)

features_selected_2.extend(['State_Factor'])

In [30]:
dict_resultados_3=entrena_lgb(data=df_train,test=df_test,features=features_selected_2,categorical=categorical,target=target)

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:17

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000475 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 587
[LightGBM] [Info] Number of data points in the train set: 53308, number of used features: 5
[LightGBM] [Info] Start training from score 83.961575
Training until validation scores don't improve for 100 rounds
[25]	valid_0's l2: 2132.39
[50]	valid_0's l2: 2163.61
[75]	valid_0's l2: 2044.16
[100]	valid_0's l2: 2031.35
Early stopping, best iteration is:
[2]	valid_0's l2: 1880.2
Fold_1
43.361330124142086

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000432 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 589
[LightGBM] [Info] Number of data points in the train set: 57449, number 

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:17


[25]	valid_0's l2: 1167.72
[50]	valid_0's l2: 1035.98
[75]	valid_0's l2: 952.03
[100]	valid_0's l2: 899.887
[125]	valid_0's l2: 867.423
[150]	valid_0's l2: 850.304
[175]	valid_0's l2: 830.133
[200]	valid_0's l2: 814.432
[225]	valid_0's l2: 802.199
[250]	valid_0's l2: 784.598
[275]	valid_0's l2: 776.852
[300]	valid_0's l2: 767.255
[325]	valid_0's l2: 761.587
[350]	valid_0's l2: 752.065
[375]	valid_0's l2: 746.718
[400]	valid_0's l2: 743.704
[425]	valid_0's l2: 741.357
[450]	valid_0's l2: 734.732
[475]	valid_0's l2: 732.489
[500]	valid_0's l2: 731.178
[525]	valid_0's l2: 726.794
[550]	valid_0's l2: 725.835
[575]	valid_0's l2: 725.119
[600]	valid_0's l2: 724.669
[625]	valid_0's l2: 726.372
[650]	valid_0's l2: 726.539
[675]	valid_0's l2: 725.751
Early stopping, best iteration is:
[589]	valid_0's l2: 723.97
Fold_2
26.90668455324115

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000431 seconds.
You can set `force_row_wise=true` to remove the over

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:17

...

[50]	valid_0's l2: 2054.94
[75]	valid_0's l2: 1991.07
[100]	valid_0's l2: 1954.87
[125]	valid_0's l2: 1922.9
[150]	valid_0's l2: 1900.5
[175]	valid_0's l2: 1884.18
[200]	valid_0's l2: 1855.49
[225]	valid_0's l2: 1851.79
[250]	valid_0's l2: 1846.95
[275]	valid_0's l2: 1837.04
[300]	valid_0's l2: 1827.29
[325]	valid_0's l2: 1818.33
[350]	valid_0's l2: 1811.89
[375]	valid_0's l2: 1803.86
[400]	valid_0's l2: 1795.67
[425]	valid_0's l2: 1784.64
[450]	valid_0's l2: 1774.87
[475]	valid_0's l2: 1769.7
[500]	valid_0's l2: 1764.65
[525]	valid_0's l2: 1759.27
[550]	valid_0's l2: 1758.33
[575]	valid_0's l2: 1752.49
[600]	valid_0's l2: 1751.47
[625]	valid_0's l2: 1746.79
[650]	valid_0's l2: 1746.76
[675]	valid_0's l2: 1743.45
[700]	valid_0's l2: 1733.95
[725]	valid_0's l2: 1732.27
[750]	valid_0's l2: 1729.06
[775]	valid_0's l2: 1727.06
[800]	valid_0's l2: 1725.5
[825]	valid_0's l2: 1723.77
[850]	valid_0's l2: 1722.79
[875]	valid_0's l2: 1717.4
[900]	valid_0's l2: 1714.34
[925]	valid_0's l2: 1709.49

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:17

[50]	valid_0's l2: 1757.71
[75]	valid_0's l2: 1703.44
[100]	valid_0's l2: 1687.99
[125]	valid_0's l2: 1637.06
[150]	valid_0's l2: 1618.72
[175]	valid_0's l2: 1601.36
[200]	valid_0's l2: 1579.78
[225]	valid_0's l2: 1571.33
[250]	valid_0's l2: 1564.39
[275]	valid_0's l2: 1554.01
[300]	valid_0's l2: 1550.75
[325]	valid_0's l2: 1540.27
[350]	valid_0's l2: 1533.61
[375]	valid_0's l2: 1527.83
[400]	valid_0's l2: 1526.17
[425]	valid_0's l2: 1524.57
[450]	valid_0's l2: 1521.67
[475]	valid_0's l2: 1513.41
[500]	valid_0's l2: 1511.1
[525]	valid_0's l2: 1503.67
[550]	valid_0's l2: 1502.66
[575]	valid_0's l2: 1503.7
[600]	valid_0's l2: 1500.9
[625]	valid_0's l2: 1500.54
[650]	valid_0's l2: 1499.21
[675]	valid_0's l2: 1499.14
[700]	valid_0's l2: 1494.23
[725]	valid_0's l2: 1490.93
[750]	valid_0's l2: 1487.75
[775]	valid_0's l2: 1483.9
[800]	valid_0's l2: 1481.06
[825]	valid_0's l2: 1479.26
[850]	valid_0's l2: 1479.01
[875]	valid_0's l2: 1478.05
[900]	valid_0's l2: 1478.18
[925]	valid_0's l2: 1476.0

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:17

[25]	valid_0's l2: 3282.1
[50]	valid_0's l2: 3030.68
[75]	valid_0's l2: 2873.36
[100]	valid_0's l2: 2798.59
[125]	valid_0's l2: 2745.95
[150]	valid_0's l2: 2719.31
[175]	valid_0's l2: 2673.17
[200]	valid_0's l2: 2664.31
[225]	valid_0's l2: 2675.42
[250]	valid_0's l2: 2679.94
[275]	valid_0's l2: 2651.65
[300]	valid_0's l2: 2650.53
[325]	valid_0's l2: 2654.7
[350]	valid_0's l2: 2620.32
[375]	valid_0's l2: 2603.45
[400]	valid_0's l2: 2594.33
[425]	valid_0's l2: 2590.99
[450]	valid_0's l2: 2581.25
[475]	valid_0's l2: 2564.71
[500]	valid_0's l2: 2557.22
[525]	valid_0's l2: 2550.24
[550]	valid_0's l2: 2547
[575]	valid_0's l2: 2532.79
[600]	valid_0's l2: 2535.88
[625]	valid_0's l2: 2548.82
[650]	valid_0's l2: 2546.32
[675]	valid_0's l2: 2541.24
Early stopping, best iteration is:
[588]	valid_0's l2: 2528.44
Fold_6
50.28354673324558

36.959649089744445
mean: 38.191635589959326
std: 7.889533343045588


In [31]:
features_selected_2

['facility_type',
 'energy_star_rating',
 'year_built',
 'floor_area',
 'State_Factor']

In [32]:
variables=['facility_type',
 'energy_star_rating',
 'year_built',
# 'floor_area',
 'State_Factor']

df_train['combination_variables']=df_train[variables].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

df_test['combination_variables']=df_test[variables].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [33]:
features_selected_3=list(features_selected_2)

features_selected_3.extend(['combination_variables'])

categorical.extend(['combination_variables'])

In [34]:
dict_resultados_4=entrena_lgb(data=df_train,test=df_test,features=features_selected_3,categorical=categorical,target=target)

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4, 5]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002112 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3973
[LightGBM] [Info] Number of data points in the train set: 53308, number of used features: 6
[LightGBM] [Info] Start training from score 83.961575
Training until validation scores don't improve for 100 rounds
[25]	valid_0's l2: 2404.95
[50]	valid_0's l2: 2400.39
[75]	valid_0's l2: 2378.34
[100]	valid_0's l2: 2442.18
Early stopping, best iteration is:
[2]	valid_0's l2: 1908.14
Fold_1
43.682252113646555

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000604 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4143
[LightGBM] [Info] Number of data points in the train set: 57449, number of used features: 6
[LightGBM] [Info] Start training from sco

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4, 5]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py

[25]	valid_0's l2: 1182.07
[50]	valid_0's l2: 1046.09
[75]	valid_0's l2: 983.79
[100]	valid_0's l2: 945.086
[125]	valid_0's l2: 912.121
[150]	valid_0's l2: 878.922
[175]	valid_0's l2: 856.241
[200]	valid_0's l2: 840.713
[225]	valid_0's l2: 822.544
[250]	valid_0's l2: 809.966
[275]	valid_0's l2: 798.952
[300]	valid_0's l2: 789.902
[325]	valid_0's l2: 782.324
[350]	valid_0's l2: 775.982
[375]	valid_0's l2: 773.013
[400]	valid_0's l2: 763.689
[425]	valid_0's l2: 757.8
[450]	valid_0's l2: 746.492
[475]	valid_0's l2: 740.927
[500]	valid_0's l2: 737.682
[525]	valid_0's l2: 735.393
[550]	valid_0's l2: 732.76
[575]	valid_0's l2: 730.787
[600]	valid_0's l2: 729.904
[625]	valid_0's l2: 728.073
[650]	valid_0's l2: 726.057
[675]	valid_0's l2: 726.27
[700]	valid_0's l2: 724.922
[725]	valid_0's l2: 723.465
[750]	valid_0's l2: 723.144
[775]	valid_0's l2: 722.907
[800]	valid_0's l2: 721.192
[825]	valid_0's l2: 721.554
[850]	valid_0's l2: 720.16
[875]	valid_0's l2: 719.817
[900]	valid_0's l2: 719.794
[

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4, 5]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py

...

[25]	valid_0's l2: 2145.76
[50]	valid_0's l2: 2060.44
[75]	valid_0's l2: 1984.11
[100]	valid_0's l2: 1943.54
[125]	valid_0's l2: 1909.44
[150]	valid_0's l2: 1885.84
[175]	valid_0's l2: 1870.94
[200]	valid_0's l2: 1851.3
[225]	valid_0's l2: 1825.83
[250]	valid_0's l2: 1812.89
[275]	valid_0's l2: 1805.38
[300]	valid_0's l2: 1788.85
[325]	valid_0's l2: 1782.12
[350]	valid_0's l2: 1775.27
[375]	valid_0's l2: 1771.83
[400]	valid_0's l2: 1767.25
[425]	valid_0's l2: 1757.23
[450]	valid_0's l2: 1753.47
[475]	valid_0's l2: 1751.73
[500]	valid_0's l2: 1748.06
[525]	valid_0's l2: 1747.05
[550]	valid_0's l2: 1737.97
[575]	valid_0's l2: 1729.79
[600]	valid_0's l2: 1725.69
[625]	valid_0's l2: 1723.02
[650]	valid_0's l2: 1720.83
[675]	valid_0's l2: 1717.22
[700]	valid_0's l2: 1713.87
[725]	valid_0's l2: 1714.4
[750]	valid_0's l2: 1713.77
[775]	valid_0's l2: 1712.95
[800]	valid_0's l2: 1708.94
[825]	valid_0's l2: 1706.45
[850]	valid_0's l2: 1706.11
[875]	valid_0's l2: 1706.46
[900]	valid_0's l2: 1707.

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4, 5]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py

[25]	valid_0's l2: 1861.35
[50]	valid_0's l2: 1766.33
[75]	valid_0's l2: 1741.9
[100]	valid_0's l2: 1713.77
[125]	valid_0's l2: 1699.59
[150]	valid_0's l2: 1692.78
[175]	valid_0's l2: 1665.43
[200]	valid_0's l2: 1648.82
[225]	valid_0's l2: 1634.72
[250]	valid_0's l2: 1621.29
[275]	valid_0's l2: 1602.54
[300]	valid_0's l2: 1590.24
[325]	valid_0's l2: 1583.63
[350]	valid_0's l2: 1574
[375]	valid_0's l2: 1563.63
[400]	valid_0's l2: 1553.47
[425]	valid_0's l2: 1551.18
[450]	valid_0's l2: 1546.57
[475]	valid_0's l2: 1535.21
[500]	valid_0's l2: 1530.97
[525]	valid_0's l2: 1524.79
[550]	valid_0's l2: 1521.97
[575]	valid_0's l2: 1517.11
[600]	valid_0's l2: 1512.26
[625]	valid_0's l2: 1510.2
[650]	valid_0's l2: 1506.3
[675]	valid_0's l2: 1503.9
[700]	valid_0's l2: 1502.59
[725]	valid_0's l2: 1501
[750]	valid_0's l2: 1496.5
[775]	valid_0's l2: 1493.55
[800]	valid_0's l2: 1492.71
[825]	valid_0's l2: 1492.21
[850]	valid_0's l2: 1490.93
[875]	valid_0's l2: 1490.43
[900]	valid_0's l2: 1493.12
[925]	

C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 4, 5]
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\anaconda3\lib\site-packages\lightgbm\basic.py

[25]	valid_0's l2: 3112.74
[50]	valid_0's l2: 2910.44
[75]	valid_0's l2: 2743.95
[100]	valid_0's l2: 2678.95
[125]	valid_0's l2: 2614.35
[150]	valid_0's l2: 2582.78
[175]	valid_0's l2: 2533.1
[200]	valid_0's l2: 2519.55
[225]	valid_0's l2: 2505.23
[250]	valid_0's l2: 2502.34
[275]	valid_0's l2: 2488.75
[300]	valid_0's l2: 2473.57
[325]	valid_0's l2: 2442.87
[350]	valid_0's l2: 2420.93
[375]	valid_0's l2: 2404.8
[400]	valid_0's l2: 2412.83
[425]	valid_0's l2: 2407.13
[450]	valid_0's l2: 2394.8
[475]	valid_0's l2: 2392.05
[500]	valid_0's l2: 2372.8
[525]	valid_0's l2: 2371.42
[550]	valid_0's l2: 2373.75
[575]	valid_0's l2: 2368.78
[600]	valid_0's l2: 2364.11
[625]	valid_0's l2: 2363.02
[650]	valid_0's l2: 2365.27
Early stopping, best iteration is:
[567]	valid_0's l2: 2358.71
Fold_6
48.566561926230115

37.286251376370544
mean: 38.29852330965076
std: 7.420688574914001


In [35]:
dict_resultados_4['importancias']

,variable,gain_1,gain_2,gain_3,gain_4,gain_5,gain_6,gain_avg
0,facility_type,3.720015e+07,4.832151e+07,5.462189e+07,5.785684e+07,6.122709e+07,6.237083e+07,5.184550e+07
1,energy_star_rating,3.026947e+07,4.611014e+07,4.805814e+07,5.677295e+07,5.740707e+07,5.668241e+07,4.772355e+07
2,floor_area,4.847139e+06,4.644579e+07,4.780648e+07,4.591617e+07,4.798364e+07,5.096891e+07,3.859984e+07
3,year_built,3.516797e+06,3.365372e+07,3.542148e+07,3.518423e+07,3.845117e+07,3.633286e+07,2.924548e+07
4,State_Factor,2.914098e+06,4.774533e+06,5.716038e+06,5.725314e+06,6.752317e+06,6.425968e+06,5.176460e+06
5,combination_variables,2.276814e+06,5.465967e+06,6.009703e+06,5.035616e+06,5.902306e+06,7.516521e+06,4.938081e+06


In [36]:
df_test['site_eui']=(dict_resultados_2['predicciones'].copy()+dict_resultados_3['predicciones'].copy())/2

In [37]:
df_test[['id','site_eui']]

,id,site_eui
0,75757,226.677474
1,75758,241.254301
2,75759,198.448890
3,75760,260.461135
4,75761,245.775285
...,...,...
9700,85457,22.348259
9701,85458,35.722786
9702,85459,30.583764
9703,85460,30.289094


In [38]:
df_test[['id','site_eui']].to_csv('submission.csv',index=False)